In [1]:
from typing import Annotated
from typing_extensions import TypedDict

import cohere
from langgraph.graph import StateGraph, add_messages
from langgraph.graph import START, END

from config_private import COHERE_API_KEY

In [2]:
MODEL_ID = "command-r-plus-08-2024"

co = cohere.ClientV2(COHERE_API_KEY)

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
def chatbot(state: State):
    print(state["messages"][-1])

    user_message = state["messages"][-1].content

    response = co.chat(
        model=MODEL_ID, messages=[{"role": "user", "content": user_message}]
    )

    return {
        "messages": state["messages"]
        + [{"role": "assistant", "content": response.message.content[0].text}]
    }

In [5]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

#### Processing input

In [ ]:
while True:
    user_input = input("Utente: ")

    if user_input.lower() in ["quit", "exit", "q"]:
        print("Arrivederci!")
        break

    state = {"messages": [{"role": "user", "content": user_input}]}

    for event in graph.stream(state):
        for value in event.values():
            print("")
            print("Assistente:", value["messages"][-1]["content"])

Utente:  Chi è Larry Ellison?


content='Chi è Larry Ellison?' additional_kwargs={} response_metadata={} id='83579a60-5018-45a3-b370-c567c6842abf'

Assistente: Larry Ellison è un imprenditore e informatico statunitense, noto per essere il co-fondatore e presidente esecutivo di Oracle Corporation, una delle più grandi aziende di software al mondo.

Ellison è nato a New York il 17 agosto 1944. Ha trascorso la sua infanzia a Chicago e, dopo la laurea, si è trasferito in California, dove ha iniziato la sua carriera nel settore tecnologico.

Nel 1977, insieme ai suoi colleghi Bob Miner e Ed Oates, Ellison ha fondato la Software Development Laboratories, che in seguito divenne Oracle Corporation. Ellison ha avuto un ruolo cruciale nello sviluppo e nella crescita dell'azienda, trasformando il database relazionale Oracle in un prodotto di successo a livello globale. Sotto la sua guida, Oracle è diventata una delle principali società di software, offrendo una gamma di prodotti e servizi per la gestione dei dati e delle inform